In [87]:
from azureml.core.authentication import AzureCliAuthentication
import azureml.core
from azureml.core import Workspace

cli_auth = AzureCliAuthentication()
ws = Workspace(subscription_id='401a2f63-c029-433e-9087-c839197089fd', resource_group='pix2para_resourcegroup', workspace_name = 'pix2para', auth=cli_auth)

In [54]:
experiment_name = 'vgg_test'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [55]:
ws.list(subscription_id = '401a2f63-c029-433e-9087-c839197089fd')

{'pix2para': [Workspace.create(name='pix2para', subscription_id='401a2f63-c029-433e-9087-c839197089fd', resource_group='pix2para_resourcegroup')]}

In [56]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
   
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    print(compute_target.get_status().serialize())

found compute target: cpu-cluster


In [8]:
import os
script_folder = os.path.join(os.getcwd(), "vggtest")
os.makedirs(script_folder, exist_ok=True)

In [13]:
from keras.applications.vgg19 import VGG19
import joblib
os.makedirs('outputs', exist_ok=True)
model = VGG19()
joblib.dump(value=model, filename='outputs/vgg.pkl')

['outputs/vgg.pkl']

In [24]:
from keras.applications.vgg19 import VGG19

model = VGG19()

In [248]:
%%writefile score.py
#right one
import json 
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19
import numpy as np
import io
from PIL import Image
import json
from numpy import array
import os
import joblib

def init():
    global model
    model = VGG19()
    
def run(image_bytes):
    image_bytes = json.loads(image_bytes)['data'] #str to string
    image_bytes = image_bytes.encode('utf-8')
    image = Image.frombytes('RGBA', (224,224), image_bytes, 'raw')
    image = image.resize((224,224),Image.ANTIALIAS)    
    image = img_to_array(image)
    image = image[:,:,0:3]
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat = model.predict(image)
    label = decode_predictions(yhat)
    label = label[0][0][1]
    #return(str(type(label)))
    return label
    
    



Overwriting score.py


In [264]:
#debugging cell: ignore
#test to check for score.py runtime errors
import base64
import json 
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19
import numpy as np
import io
from PIL import Image
import json
from numpy import array
import struct

def run(image_bytes):
    
    #data = np.array(image)
    # make prediction
    print(type((image_bytes)))
    #print(image_bytes[0:100])
    image_bytes = image_bytes.encode()
    #print(type((image_bytes)))
    print(image_bytes[0:100])
    
    image_bytes = json.loads(image_bytes)['data']
    print(type((image_bytes)))
    #image_bytes = struct.unpack('d', image_bytes)[0]
    print(type((image_bytes)))
    image_bytes = image_bytes.encode('utf-8')
    print(type((image_bytes)))
    encode_len = len(image_bytes)
    print(encode_len)
    
    image = Image.frombytes('RGBA', (263,128), image_bytes, 'raw')
    print(type(image))
    image = image.resize((224,224),Image.ANTIALIAS)
    pilimg = list(Image.open(image_path, 'r').getdata())
    #pix_val_flat = [x for sets in pilimg for x in sets]
    print(pix_val_flat[0:10])
    image.show()
    
    image = img_to_array(image)
    image = image[:,:,0:3]
    print(image.shape)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    
    model = VGG19()
    yhat = model.predict(image)
    label = decode_predictions(yhat)
    label = label[0][0][1]
    print(type(label))
    return label


'''image_path = '/Users/audreycui01/New Unity Project/assets/screenshot.png'
image = Image.open(image_path, 'r')
#image.show()
pilimg = list(image.getdata())
#pix_val_flat = [x for sets in pilimg for x in sets]
print(pilimg[0:100])
print(array(pilimg).shape)
#image_bytes = base64.b64encode(pix_val_flat)
image_bytes = bytearray(pix_val_flat)
image_bytes = str(image_bytes)
#image_bytes = image_bytes.decode('utf-8')'''




with open(image_path, "rb") as imageFile:
    image_bytes = bytes(imageFile.read())
    #image_bytes = image_bytes.decode('utf-8')
#image_bytes = image_bytes.encode('utf-8')
#image = Image.frombytes('RGBA', (224,224), image_bytes, 'raw')

#decode_len = len(image_bytes)
#print(decode_len)
#data = {"data": [image_bytes]}
#input_data = json.dumps(data)
#print(input_data[0:100])

scorejson_path = '/Users/audreycui01/New Unity Project/assets/scoreJson.txt'
with open(scorejson_path, "rb") as sj: 
    scorejson = sj.read()
scorejson = scorejson.decode()
label = run(scorejson)
print(label)

<class 'str'>
b'{"data":"iVBORw0KGgoAAAANSUhEUgAABSIAAAPYCAYAAADO8YdiAAAgAElEQVR4Aay9W5IeSbKkh0t1833IVfCZT1zeWezZwAh'
<class 'str'>
<class 'str'>
<class 'bytes'>
1823572
<class 'PIL.Image.Image'>
[130, 130, 130, 255, 130, 130, 130, 255, 104, 104]
(224, 224, 3)
<class 'str'>
poncho


In [22]:
from azureml.core.model import Model

model_name = "vggtest"


In [31]:
import os
model = Model.register(model_path="outputs/vgg.pkl",
                        model_name=model_name,
                        description="vgg test",
                        workspace=ws)
#model.download(target_dir=os.getcwd(), exist_ok=True)

Registering model vggtest


In [57]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("scikit-learn==0.22.1")
myenv.add_pip_package("keras.applications")
myenv.add_pip_package("keras.preprocessing")
myenv.add_pip_package("azureml-defaults")
myenv.add_pip_package("keras")
myenv.add_pip_package("tensorflow")
myenv.add_pip_package("numpy")
myenv.add_pip_package("pillow")


with open("my_vgg_env.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [58]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=8, 
                                               description='vggtest')

In [255]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="my_vgg_env", file_path="my_vgg_env.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='vggtest', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Running..............................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 1.17 s, sys: 185 ms, total: 1.35 s
Wall time: 5min 52s


In [254]:
service.delete()

In [83]:
service = AciWebservice(ws, 'vggtest')

inference_config = InferenceConfig(entry_script="score.py", environment=myenv)
myenv = Environment.from_conda_specification(name="my_vgg_env", file_path="my_vgg_env.yml")

service.update(inference_config = inference_config)
service.wait_for_deployment(show_output=True)

In [256]:
print(service.scoring_uri)
print(service.state)


http://17ebffef-bd8b-43dc-bb83-4649983647cc.westus.azurecontainer.io/score
Healthy
